## Dynamic QCNNs

This notebook shows how to use the `dynamic_qcnn` package to generate Quantum Convolutional Neural Networks (QCNNs) using [cirq](https://quantumai.google/cirq) and [tensorflow quantum](https://www.tensorflow.org/quantum/overview). The core functionality is two primitive operations or cells (`QConv` for convolutions and `QPool` for pooling) which can be dynamically stacked ontop each to create a full QCNN. Then there are functions like `binary_tree_r` to generate families QCNNs using this core functionality. The tool helps with the following:
 - **Architecture search space design:** it's easy to define and generate families of QCNNs that capture different design motifs such that a system can greedily/intelligently search through them.
 - **Accessibility and usability of the QCNN:** the tool abstracts away a lot of details enabling different levels of interaction for QCNN modelling. It's also library agnostic and can be used with any QML library.

The example shown is a binary classification model that distinguishes between two musical genres using the well known GTZAN dataset.

___
*A cute robot building itself with artifical intelligence, pencil drawing -  generated with* [Dall$\cdot$E 2](https://openai.com/dall-e-2/)

<img src="../img/DALL·E 2022-08-17 11.48.32 - A cute robot building itself with artifical intelligence, pencil drawing.png" alt="drawing" width="200"/>

In [ ]:
# Load libraries
import sympy
import numpy as np
from collections import namedtuple
import pandas as pd
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
from tensorflow import keras
from sklearn.model_selection import train_test_split

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

### Experimental setup

The dataset contains statistics from 1000 audio tracks, each being a 30-second recording of some song. Each song is given a label of one of the following ten musical genres: **blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock**. See [marsyas](https://github.com/marsyas/website/blob/master/downloads/data-sets.rst) and [kaggle](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification) for more info.

We'll build a model to distinguish **rock** from **reggae**

In [ ]:
# Specify data path
path = "../data/gtzan_30s_stats.csv"
# Specify genres to build classification model from, options are:
# blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock
target_pair = ["rock", "reggae"]
# Read data
raw = pd.read_csv(path)
raw.head()

In [ ]:
# Specify target column
target = "label"
# Specify columns to remove
columns_to_remove = ["filename", "length", target]

For the data cleaning component we split the data into a test and train set and remove unnecesary columns. 

In [ ]:
# Seperate X (features) and y (target) from dataset
y = raw.loc[:, target]
X = raw.drop(columns_to_remove, axis=1)
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
)
# Use a named typle to keep track of the changes to train and test samples
Samples = namedtuple("Samples", ["X_train", "y_train", "X_test", "y_test"])
# samples_raw is an instance of Samples, containing the raw samples
# access train features by samples_raw.X_train
samples_raw = Samples(X_train, y_train, X_test, y_test)

Next we filter out all genres except those specified by `target_pair`.

In [ ]:
# Filter out unneccesary data, only store songs with labels specified in target_pair
train_filter = np.where(
    (samples_raw.y_train == target_pair[0]) | (samples_raw.y_train == target_pair[1])
)
test_filter = np.where(
    (samples_raw.y_test == target_pair[0]) | (samples_raw.y_test == target_pair[1])
)
X_train_filtered, X_test_filtered = (
    samples_raw.X_train.iloc[train_filter],
    samples_raw.X_test.iloc[test_filter],
)
y_train_filtered, y_test_filtered = (
    samples_raw.y_train.iloc[train_filter],
    samples_raw.y_test.iloc[test_filter],
)
# Convert target to binary int values, (genre_1, genre_2)->(0,1)
y_train_filtered = np.where(y_train_filtered == target_pair[1], 1, 0)
y_test_filtered = np.where(y_test_filtered == target_pair[1], 1, 0)
# samples_filtered now contains the latest X, y train and test data
samples_filtered = Samples(
    X_train_filtered, y_train_filtered, X_test_filtered, y_test_filtered
)

### Preprocessing

**Feature Scaling and Selection**

Here we manually select $8$ features to build the model on and then scale them to range between $[0,\frac{\pi}{2}]$. The selection can be automated with strategies like PCA or tree based methods using [sklearn pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Even though we just scale the data, the code is presented in such a way so that it's easy to add other pipeline steps.

In [ ]:
# Specify features to build model on
features = [
    "mfcc2_var",
    "mfcc3_var",
    "mfcc4_var",
    "mfcc5_var",
    "mfcc7_var",
    "mfcc8_var",
    "mfcc11_mean",
    "mfcc13_mean",
]
X_train_selected = np.array(samples_filtered.X_train[features])
X_test_selected = np.array(samples_filtered.X_test[features])

samples_selected = Samples(
    X_train_selected, samples_filtered.y_train, X_test_selected, samples_filtered.y_test
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Create a pipeline_list which will contain preprocessing steps
pipeline_list = []
# For now we only scale the data, but more complicated pipelines can be constructed with this pattern
scaler = (
    "minmax",
    MinMaxScaler(feature_range=[0, np.pi / 2]),
)
pipeline_list.append(scaler)
pipeline = Pipeline(pipeline_list)
# Fit pipeline
pipeline.fit(samples_selected.X_train, samples_selected.y_train)
# Transform data
X_train_tfd = pipeline.transform(samples_selected.X_train)
X_test_tfd = pipeline.transform(samples_selected.X_test)
samples_tfd = Samples(
    X_train_tfd, samples_selected.y_train, X_test_tfd, samples_selected.y_test
)

### Encode data into a quantum state

In [ ]:
def qubit_encoding(x, gate=cirq.rx):
    circuit = cirq.Circuit()
    for i, value in enumerate(x):
        qubit = cirq.LineQubit(i)
        circuit.append(gate(value).on(qubit))

    return circuit


X_train_encoded = tfq.convert_to_tensor(
    [qubit_encoding(x) for x in samples_tfd.X_train]
)
X_test_encoded = tfq.convert_to_tensor([qubit_encoding(x) for x in samples_tfd.X_test])

samples_encoded = Samples(
    X_train_encoded, samples_tfd.y_train, X_test_encoded, samples_tfd.y_test
)

### Build model

Before building the model, we'll showcase some of the functionality through examples. For illustration purposes we use a simple $CR_z(\theta)$ gate for convolutions and a CNOT for pooling. These are chosen arbitrarily, when using the tool the user can to send a function containing the sequence of gates constituting a convolution or pooling operation. This function may be built with any QML library such as `Cirq`, `Qiskit` or `Pennylane`.

In [ ]:
from cirq.contrib.svg import SVGCircuit

# Default Convolution
bits = (1, 2)
symbols = sympy.symbols("x_0:2")
circuit = cirq.Circuit()
q0, q1 = cirq.LineQubit(bits[0]), cirq.LineQubit(bits[1])
circuit += cirq.rz(symbols[0]).on(q1).controlled_by(q0)
print("Convolution unitary:")
SVGCircuit(circuit)

In [ ]:
bits = (1, 2)
circuit = cirq.Circuit()
q0, q1 = cirq.LineQubit(bits[0]), cirq.LineQubit(bits[1])
circuit += cirq.CNOT(q0, q1)
print("Pooling unitary:")
SVGCircuit(circuit)

In [ ]:
from dynamic_qcnn import Qcnn_cirq as Qcnn_cirq
from dynamic_qcnn import (
    Qcnn,
    Qfree,
    Qconv,
    Qpool,
    Qdense,
    binary_tree_r,
    convert_graph_to_circuit_cirq,
    plot_graph,
    pretty_cirq_plot,
)

In [ ]:
### Reverse binary tree
N = 8
# level 1
m1_1 = Qconv(1)
m1_2 = Qpool(filter="outside")
# level 2
m2_1 = m1_1 + m1_2
# level 3
m3_1 = Qfree(N) + m2_1 * int(np.log2(N))

circuit, symbols = convert_graph_to_circuit_cirq(m3_1)
# m3_1 + Qfree([1,3,5]) + Qconv(5)+m3_1)*4
SVGCircuit(circuit)

Scale up the same model, and alternate architectures

In [ ]:
import cirq

# Default pooling circuit
def V(bits, symbols=None):
    circuit = cirq.Circuit()
    q0, q1 = cirq.LineQubit(bits[0]), cirq.LineQubit(bits[1])
    circuit += cirq.rz(symbols[0]).on(q1).controlled_by(q0)
    circuit += cirq.X(q0)
    circuit += cirq.rx(symbols[1]).on(q1).controlled_by(q0)
    return circuit


# Default convolution circuit
def U(bits, symbols=None):
    circuit = cirq.Circuit()
    q0, q1 = cirq.LineQubit(bits[0]), cirq.LineQubit(bits[1])
    circuit += cirq.rx(symbols[0]).on(q0)
    circuit += cirq.rx(symbols[1]).on(q1)
    circuit += cirq.rz(symbols[2]).on(q0)
    circuit += cirq.rz(symbols[3]).on(q1)
    circuit += cirq.rz(symbols[4]).on(q1).controlled_by(q0)
    circuit += cirq.rz(symbols[5]).on(q0).controlled_by(q1)
    circuit += cirq.rx(symbols[6]).on(q0)
    circuit += cirq.rx(symbols[7]).on(q1)
    circuit += cirq.rz(symbols[8]).on(q0)
    circuit += cirq.rz(symbols[9]).on(q1)
    return circuit

In [ ]:
# Currently not implemented
# #from dynamic_qcnn import Qcnn_cirq
# m  = Qfree(8) + (Qconv(1) + Qpool(filter="inside"))*3

# readout = cirq.LineQubit(m.head.Q_avail[0])
# circuit, symbols = convert_graph_to_circuit_cirq(m)

# model = tf.keras.Sequential(
#     [
#         # The Qcnn layer returns the expected value of the readout gate, range [-1,1]. By default readout is criq.Z and the model determines
#         # which qubit to measure based on the one that's left over
#         Qcnn_cirq(circuit=circuit, symbols=symbols, readout=readout),
#         # Convert expectation values to lie between 0 and 1
#         tf.keras.layers.Rescaling(1.0 / 2, offset=0.5),
#     ]
# )

In [ ]:
# model.compile(
#     optimizer="Adam",
#     loss="binary_crossentropy",
#     metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)],
# )
# # model.run_eagerly = True
# model.fit(x=samples_encoded.X_train, y=samples_encoded.y_train, epochs=100)

### Evaluate

In [ ]:
# model.summary()
# print(model.trainable_variables)

# qcnn_results = model.evaluate(samples_encoded.X_test, samples_encoded.y_test)
# # results.append([f"{s_c}_{s_p}_{pool_filter}", qcnn_results])
# print(qcnn_results)